# Project Proposal

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Possible questions for Starbucks data:
* Classification: Can we use selected nutritional values of a Starbucks drink to predict its beverage category or beverage type?
* Regression: Can we use the total fat in a Starbucks drink to predict its calories?
* Regression: Can we use the total carbs in a Starbucks food item to predict its calories?

Possible questions for Salaries data:
* Classification: Can we use starting median salary and mid-career median salary to predict the region of education?
* Regression: Can we use starting median salary to predict mid-career median salary?

In [11]:
drink_data <- read_csv("data/starbucks/drinkMenu.csv")
food_data <- read_csv("data/starbucks/foodMenu.csv")
head(drink_data)

Parsed with column specification:
cols(
  Beverage_category = col_character(),
  Beverage = col_character(),
  Beverage_prep = col_character(),
  Calories = col_double(),
  `Total Fat (g)` = col_character(),
  `Trans Fat (g)` = col_double(),
  `Saturated Fat (g)` = col_double(),
  `Sodium (mg)` = col_double(),
  `Total Carbohydrates (g)` = col_double(),
  `Cholesterol (mg)` = col_double(),
  `Dietary Fibre (g)` = col_double(),
  `Sugars (g)` = col_double(),
  `Protein (g)` = col_double(),
  `Vitamin A (% DV)` = col_character(),
  `Vitamin C (% DV)` = col_character(),
  `Calcium (% DV)` = col_character(),
  `Iron (% DV)` = col_character(),
  `Caffeine (mg)` = col_character()
)

Warning message:
“Missing column names filled in: 'X1' [1]”
Parsed with column specification:
cols(
  X1 = col_character(),
  Calories = col_double(),
  `Fat (g)` = col_double(),
  `Carb. (g)` = col_double(),
  `Fiber (g)` = col_double(),
  `Protein (g)` = col_double()
)



Beverage_category,Beverage,Beverage_prep,Calories,Total Fat (g),Trans Fat (g),Saturated Fat (g),Sodium (mg),Total Carbohydrates (g),Cholesterol (mg),Dietary Fibre (g),Sugars (g),Protein (g),Vitamin A (% DV),Vitamin C (% DV),Calcium (% DV),Iron (% DV),Caffeine (mg)
<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>
Coffee,Brewed Coffee,Short,3,0.1,0.0,0.0,0,5,0,0,0,0.3,0%,0%,0%,0%,175
Coffee,Brewed Coffee,Tall,4,0.1,0.0,0.0,0,10,0,0,0,0.5,0%,0%,0%,0%,260
Coffee,Brewed Coffee,Grande,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,0%,0%,330
Coffee,Brewed Coffee,Venti,5,0.1,0.0,0.0,0,10,0,0,0,1.0,0%,0%,2%,0%,410
Classic Espresso Drinks,Caffè Latte,Short Nonfat Milk,70,0.1,0.1,0.0,5,75,10,0,9,6.0,10%,0%,20%,0%,75
Classic Espresso Drinks,Caffè Latte,2% Milk,100,3.5,2.0,0.1,15,85,10,0,9,6.0,10%,0%,20%,0%,75


In [10]:
byRegion <- read_csv("data/salaries/byRegion.csv")
byCollege <- read_csv("data/salaries/byCollege.csv")
byDegree <- read_csv("data/salaries/byDegree.csv")
head(byRegion)

Parsed with column specification:
cols(
  `School Name` = col_character(),
  Region = col_character(),
  `Starting Median Salary` = col_character(),
  `Mid-Career Median Salary` = col_character(),
  `Mid-Career 10th Percentile Salary` = col_character(),
  `Mid-Career 25th Percentile Salary` = col_character(),
  `Mid-Career 75th Percentile Salary` = col_character(),
  `Mid-Career 90th Percentile Salary` = col_character()
)

Parsed with column specification:
cols(
  `School Name` = col_character(),
  `School Type` = col_character(),
  `Starting Median Salary` = col_character(),
  `Mid-Career Median Salary` = col_character(),
  `Mid-Career 10th Percentile Salary` = col_character(),
  `Mid-Career 25th Percentile Salary` = col_character(),
  `Mid-Career 75th Percentile Salary` = col_character(),
  `Mid-Career 90th Percentile Salary` = col_character()
)

Parsed with column specification:
cols(
  `Undergraduate Major` = col_character(),
  `Starting Median Salary` = col_character(),
  `Mid-Car

School Name,Region,Starting Median Salary,Mid-Career Median Salary,Mid-Career 10th Percentile Salary,Mid-Career 25th Percentile Salary,Mid-Career 75th Percentile Salary,Mid-Career 90th Percentile Salary
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
Stanford University,California,"$70,400.00","$129,000.00","$68,400.00","$93,100.00","$184,000.00","$257,000.00"
California Institute of Technology (CIT),California,"$75,500.00","$123,000.00",N/A,"$104,000.00","$161,000.00",N/A
Harvey Mudd College,California,"$71,800.00","$122,000.00",N/A,"$96,000.00","$180,000.00",N/A
"University of California, Berkeley",California,"$59,900.00","$112,000.00","$59,500.00","$81,000.00","$149,000.00","$201,000.00"
Occidental College,California,"$51,900.00","$105,000.00",N/A,"$54,800.00","$157,000.00",N/A
Cal Poly San Luis Obispo,California,"$57,200.00","$101,000.00","$55,000.00","$74,700.00","$133,000.00","$178,000.00"
